<a href="https://colab.research.google.com/github/Valeron31/testrepo/blob/master/JupyterDashAppSpaceX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pip3 install pandas dash
#wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

#wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

#python3 spacex_dash_app.py

!pip install dash==1.19.0
!pip install jupyter_dash
!pip install --upgrade plotly


# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash
import plotly.graph_objects as graph_objects

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', 
                                options=[{'label':'All Sites', 'value':'All'},
                                {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                                {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'},
                                {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},
                                {'label':'KSC LC-39A','value':'KSC SLC-39A'}],
                                value='All',
                                placeholder="Select a Launch Site here",
                                searchable = True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000,
                                marks = {0:'0',
                                2500:'2500',
                                5000:'5000',
                                7500:'7500'},
                                value=[min_payload,max_payload]),
                                html.Br(),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                html.Br(),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    df = spacex_df
    if entered_site == 'All':
        fig = px.pie(df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        df=spacex_df[spacex_df['Launch Site']==entered_site]
        filtered_df=df.groupby('class').size().reset_index(name = 'class count')
        fig=px.pie(filtered_df, values='class count', names='class',
        title='Total Success Launches By %s' % entered_site)
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value')])
def get_scatter_plot(entered_site, payload):
    mask = spacex_df[(spacex_df['Payload Mass (kg)'] > payload[0]) & (spacex_df['Payload Mass (kg)'] < payload[1])]
    if entered_site == 'All':
        fig = px.scatter(mask, x="Payload Mass (kg)", y="class", 
                         color="Booster Version Category",
                         title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        #return the outcome scatter plot for the selected site
        mask_filtered = mask[mask['Launch Site'] == entered_site]
        #mask2 = (df2['Payload Mass (kg)'] > low) & (df2['Payload Mass (kg)'] < high)
        fig = px.scatter(mask_filtered, x="Payload Mass (kg)", y="class", 
                         color="Booster Version Category",
                         title='Correlation between Payload and Success for %s' % entered_site)
        return fig   
# Run the app
if __name__ == '__main__':
    app.run_server(mode='inline', port = 8090 ,dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)


     |████████████████████████████████| 75 kB 3.7 MB/s 
     |████████████████████████████████| 1.0 MB 51.4 MB/s 
     |████████████████████████████████| 3.5 MB 34.1 MB/s 
     |████████████████████████████████| 188 kB 60.1 MB/s 
     |████████████████████████████████| 1.8 MB 49.0 MB/s 
     |████████████████████████████████| 357 kB 53.7 MB/s 
  Created wheel for dash: filename=dash-1.19.0-py3-none-any.whl size=84021 sha256=73f1f725fde7eb1ee67933cf3ef6d5505eca4499e63d5b5fb269892e528e0101
  Stored in directory: /root/.cache/pip/wheels/85/e9/f4/033d9bd19fe04ba8b3b41b6cfaec3c4634e85bdcdd82858ff3
  Created wheel for dash-core-components: filename=dash_core_components-1.15.0-py3-none-any.whl size=3527013 sha256=0cd65ec3f85d491d2eea59c6c32ce1d891935b4ce25285f9e7c07cde26771661
  Stored in directory: /root/.cache/pip/wheels/02/a5/de/374685270e68a4a49867f9c0ddd477a85e09eb1a2cb0998f92
  Created wheel for dash-html-components: filename=dash_html_components-1.1.2-py3-none-any.whl size=427844 sha25

<IPython.core.display.Javascript object>